In [1]:
# AI-Driven Sales Optimizer

# This notebook demonstrates the implementation of a Bayesian Optimization model to enhance sales performance and cost management for businesses. We'll use historical sales data to provide data-driven insights and automate decision-making in sales processes.

# python
# Import necessary libraries
import pandas as pd
import numpy as np
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import json
from datetime import datetime

print("Libraries imported successfully.")


SyntaxError: unterminated string literal (detected at line 3) (165574021.py, line 3)

In [ ]:
# Load sales dataset
sales_data = pd.read_csv('sales_data.csv')

# Display first few rows and basic info
print("First few rows of the dataset:")
print(sales_data.head())
print("\nDataset info:")
print(sales_data.info())

# Basic statistical summary
print("\nStatistical summary of numerical columns:")
print(sales_data.describe())

In [ ]:
# Calculate profit
sales_data['Profit'] = sales_data['Revenue'] - sales_data['Cost']

print("Profit calculated. Updated dataset head:")
print(sales_data.head())

# Check for any missing values
print("\nMissing values in the dataset:")
print(sales_data.isnull().sum())

# If there are missing values, you might want to handle them here
# For example: sales_data = sales_data.dropna()

In [ ]:
# Define objective function for Bayesian Optimization
@use_named_args(dimensions=[Real(0, sales_data['Sales'].max(), name='sales_target'), 
                            Real(0, sales_data['Cost'].max(), name='cost_target')])
def sales_optimization(sales_target, cost_target):
    sales_diff = np.abs(sales_data['Sales'] - sales_target).sum()
    cost_diff = np.abs(sales_data['Cost'] - cost_target).sum()
    profit = sales_data['Profit'].sum()
    return -(profit - (sales_diff + cost_diff))

print("Optimization function defined.")

In [ ]:
# Run Bayesian Optimization
result = gp_minimize(sales_optimization, 
                     dimensions=[Real(0, sales_data['Sales'].max()), 
                                 Real(0, sales_data['Cost'].max())],
                     n_calls=100, 
                     random_state=42)

print("Optimization complete. Printing results...")
print(f"Optimized Sales Target: ${result.x[0]:,.2f}")
print(f"Optimized Cost Target: ${result.x[1]:,.2f}")
print(f"Estimated Profit: ${-result.fun:,.2f}")

In [ ]:
current_profit = sales_data['Profit'].sum()
optimized_profit = -result.fun
improvement = (optimized_profit - current_profit) / current_profit * 100

print(f"Current Total Profit: ${current_profit:,.2f}")
print(f"Optimized Total Profit: ${optimized_profit:,.2f}")
print(f"Potential Improvement: {improvement:.2f}%")

print("\nInterpretation:")
print(f"1. By targeting sales of ${result.x[0]:,.2f} and costs of ${result.x[1]:,.2f}, ")
print(f"   we could potentially increase our profit by {improvement:.2f}%.")
print("2. This optimization suggests we should focus on high-value sales while keeping costs under control.")
print("3. The model provides a clear, data-driven target for our sales and finance teams to work towards.")

In [ ]:
# Visualization
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(sales_data['Sales'], 
                     sales_data['Cost'], 
                     sales_data['Revenue'], 
                     c=sales_data['Profit'], 
                     cmap='viridis')
ax.set_xlabel('Sales')
ax.set_ylabel('Cost')
ax.set_zlabel('Revenue')
plt.colorbar(scatter, label='Profit')
plt.title("Sales, Cost, Revenue, and Profit Distribution")

# Add optimized point
ax.scatter([result.x[0]], [result.x[1]], [optimized_profit], color='red', s=100, label='Optimized Point')

plt.legend()
plt.show()

print("Visualization Interpretation:")
print("1. The scatter plot shows the relationship between Sales, Cost, Revenue, and Profit.")
print("2. Each point represents a sale, with the color indicating the profit (darker colors = higher profit).")
print("3. The red point shows our optimized target, demonstrating where we should aim for maximum profit.")
print("4. This visualization can help identify patterns and outliers in our sales data.")

In [ ]:
# Save results
results = {
    "sales_target": float(result.x[0]),
    "cost_target": float(result.x[1]),
    "estimated_profit": float(-result.fun),
    "improvement_percentage": float(improvement)
}
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"optimization_results_{timestamp}.json"

with open(filename, "w") as f:
    json.dump(results, f, indent=2)

print(f"Results saved to {filename}")
print("\nNext Steps:")
print("1. Implement these optimized targets in our sales strategy.")
print("2. Monitor performance and compare against the estimated improvement.")
print("3. Regularly re-run this optimization with updated data to ensure we're always targeting optimal performance.")
print("4. Consider incorporating more features (e.g., seasonality, product categories) for more nuanced optimization.")